# Import Libraries & methods

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import shutil
import random

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten, Input, BatchNormalization, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications.vgg19 import preprocess_input

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

from copy import deepcopy
from datetime import datetime


2024-12-09 07:10:55.702367: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 07:10:56.630939: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# FedProx Algorithm

In [2]:
def loss_classifier(predictions, labels):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=predictions)
    return tf.reduce_mean(loss)


def loss_dataset(model, dataset, loss_f):
    loss = 0
    for idx, (features, labels) in enumerate(dataset):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        loss += loss_f(predictions, labels_tf)
    loss /= (idx + 1)
    return loss


def accuracy_dataset(model, dataset):
    correct = 0
    total = 0
    for features, labels in dataset:
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)
        correct += tf.reduce_sum(tf.cast(tf.equal(predicted, labels_tf), tf.int32)).numpy()
        total += labels_tf.shape[0]

    accuracy = 100 * correct / total
    return accuracy


def train_step(model, model_0, mu, optimizer, train_data, loss_f):
    total_loss = 0
    for idx, (features, labels) in enumerate(train_data):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        with tf.GradientTape() as tape:
            predictions = model(features_tf)
            loss = loss_f(predictions, labels_tf)
            loss += mu / 2 * difference_models_norm_2(model, model_0)

        total_loss += loss
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss / (idx + 1)


def local_learning(model, mu, optimizer, train_data, epochs, loss_f):
    # Clone the model instead of using deepcopy
    model_0 = tf.keras.models.clone_model(model)
    model_0.set_weights(model.get_weights())  # Set weights to be identical initially

    for e in range(epochs):
        local_loss = train_step(model, model_0, mu, optimizer, train_data, loss_f)

    return local_loss


def difference_models_norm_2(model_1, model_2):
    norm = tf.reduce_sum([tf.reduce_sum(tf.square(w1 - w2)) for w1, w2 in zip(model_1.trainable_variables, model_2.trainable_variables)])
    return norm

def set_to_zero_model_weights(model):
    for layer_weights in model.trainable_variables:
        layer_weights.assign(tf.zeros_like(layer_weights))

def average_models(model, clients_models_hist, weights):
    set_to_zero_model_weights(model)
    for k, client_hist in enumerate(clients_models_hist):
        for idx, layer_weights in enumerate(model.trainable_variables):
            contribution = client_hist[idx] * weights[k]
            layer_weights.assign_add(contribution)

#_______________________________________________________

In [3]:
def FedProx(model, training_sets, n_iter, testing_sets, mu=0.0, epochs=1, lr=0.0001, decay=1):
    # Verify that `model` is a Keras model instance
    if not isinstance(model, tf.keras.Model):
        raise TypeError("The provided model is not a TensorFlow Keras model. Please provide a valid Keras model.")

    loss_f = loss_classifier
    K = len(training_sets)
    n_samples = sum([len(db) for db in training_sets])
    weights = [len(db) / n_samples for db in training_sets]
    print("Clients' weights:", weights)

    # Initialize history lists for training and testing
    train_loss_hist = []
    train_acc_hist = []
    test_loss_hist = []
    test_acc_hist = []
    models_hist = []

    for i in range(n_iter):
        clients_params = []
        clients_losses = []
        clients_accuracies = []

        for k in range(K):
            # Clone the model and set weights for local training
            local_model = tf.keras.models.clone_model(model)
            local_model.set_weights(model.get_weights())

            local_optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

            # Perform local training and track the loss
            local_loss = local_learning(local_model, mu, local_optimizer, training_sets[k], epochs, loss_f)
            clients_losses.append(local_loss)

            # Track training accuracy for the client
            train_acc = accuracy_dataset(local_model, training_sets[k])
            clients_accuracies.append(train_acc)

            # Store model parameters (deep copy to ensure immutability)
            clients_params.append([tf.identity(tens_param) for tens_param in local_model.trainable_variables])

        # Average the local models into the global model
        average_models(model, clients_params, weights=weights)
        models_hist.append(deepcopy(clients_params))

        # Collect metrics for this iteration
        train_loss_hist.append(clients_losses)
        train_acc_hist.append(clients_accuracies)

        # Compute testing metrics using the global model
        test_loss_hist.append([loss_dataset(model, dl, loss_f).numpy() for dl in testing_sets])
        test_acc_hist.append([accuracy_dataset(model, dl) for dl in testing_sets])

        # Update learning rate by decay factor
        lr *= decay
        print(f'====> i: {i+1} Server Test Accuracy: {test_acc_hist[-1]}')

    return model, train_loss_hist, train_acc_hist, test_loss_hist, test_acc_hist



# To save model metrics

In [5]:
# Save the history to a file with detailed formatting
def save_history_to_file(filename, train_loss, train_acc, test_loss, test_acc):
    with open(filename, "w") as f:
        f.write("FedProx Training and Testing Metrics\n")
        f.write("=" * 50 + "\n")
        
        for i in range(len(train_loss)):
            try:
                # Ensure data is printed in the format shown, converting Tensors to their numpy values if needed
                train_loss_str = [float(tensor.numpy()) if hasattr(tensor, 'numpy') else float(tensor) for tensor in train_loss[i]]
                train_acc_str = [float(val) for val in train_acc[i]]
                test_loss_str = [float(val) for val in test_loss[i]]
                test_acc_str = [float(val) for val in test_acc[i]]
                
                f.write(f"Iteration {i + 81}:\n")
                f.write(f"Train Loss: {train_loss_str}\n")
                f.write(f"Train Accuracy: {train_acc_str}\n")
                f.write(f"Test Loss: {test_loss_str}\n")
                f.write(f"Test Accuracy: {test_acc_str}\n")
                f.write("-" * 50 + "\n")
                
            except (ValueError, TypeError) as e:
                f.write(f"Error processing round {i + 81}: {e}\n")
                f.write("-" * 50 + "\n")
    
    print(f"History saved to {filename}")


# Load training and testing data

In [6]:
# Directories for each client
split_dirs = [
    "Federated_Learning_NON_IID/Model_1",
    "Federated_Learning_NON_IID/Model_2",
    "Federated_Learning_NON_IID/Model_3",
    "Federated_Learning_NON_IID/Model_4"
]

# Number of clients
n_clients = len(split_dirs)

# Updated image transformations: Normalize first, then permute
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

def get_tumor_dataloaders(split_dirs, batch_size=25, shuffle=True):
    """
    Returns data loaders for all clients for both training and testing sets.
    """
    tumor_iid_train_dls = []
    tumor_iid_test_dls = []

    for client_idx, client_dir in enumerate(split_dirs):
        # Get the directory for the current client
        train_dir = os.path.join(client_dir, 'train')
        test_dir = os.path.join(client_dir, 'test')

        # Check if the directories exist
        if not os.path.exists(train_dir) or not os.path.exists(test_dir):
            print(f"Directory not found for client {client_idx + 1}:")
            print(f"Train dir: {train_dir}")
            print(f"Test dir: {test_dir}")
            continue

        # Load training data for the current client
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

        # Load testing data for the current client
        test_dataset = datasets.ImageFolder(test_dir, transform=transform)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Append the dataloaders for the current client to the list
        tumor_iid_train_dls.append(train_loader)
        tumor_iid_test_dls.append(test_loader)

    return tumor_iid_train_dls, tumor_iid_test_dls

# Get the training and testing data loaders
Tumor_iid_train_dls, Tumor_iid_test_dls = get_tumor_dataloaders(split_dirs, batch_size=25)

# Checking the sizes of the images in the data loaders to verify the shape
for batch_idx, (images, labels) in enumerate(Tumor_iid_train_dls[0]):  # Checking for client 1
    print(f"Batch {batch_idx} image sizes: {images.size()}")  # Should print torch.Size([25, 224, 224, 3])
    break  # Check only the first batch


Batch 0 image sizes: torch.Size([25, 224, 224, 3])


# ResNet50 + CBAM model 

In [7]:
# ---------------- CBAM Block Definition ------------------
def cbam_block(inputs, reduction_ratio=0.5):
    channels = inputs.shape[-1]

    # Channel attention
    avg_pool = layers.GlobalAveragePooling2D()(inputs)
    max_pool = layers.GlobalMaxPooling2D()(inputs)
    shared_layer_1 = layers.Dense(int(channels * reduction_ratio), activation='relu', use_bias=True)
    shared_layer_2 = layers.Dense(channels, activation='relu', use_bias=True)

    avg_pool = shared_layer_1(avg_pool)
    avg_pool = shared_layer_2(avg_pool)
    max_pool = shared_layer_1(max_pool)
    max_pool = shared_layer_2(max_pool)

    attention = layers.Add()([avg_pool, max_pool])
    attention = layers.Activation('sigmoid')(attention)
    attention = layers.Reshape((1, 1, channels))(attention)
    scaled_inputs = layers.Multiply()([inputs, attention])

    # Spatial attention
    squeeze = layers.Conv2D(filters=1, kernel_size=1, activation='sigmoid', use_bias=False)(scaled_inputs)
    expanded_inputs = layers.Multiply()([scaled_inputs, squeeze])

    return expanded_inputs

# ---------------- Model Definitions ------------------

# ResNet50 with CBAM and a more complex output structure
def ResNet50_CBAM_Model():
    input_shape = (224, 224, 3)
    resnet50 = tf.keras.applications.ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')

    x = layers.GlobalAveragePooling2D()(resnet50.output)
    x = layers.Dense(units=1024, activation='relu')(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = layers.Dense(units=256, activation='relu')(x)
    x = layers.Dense(units=128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Reshape((1, 1, 128))(x)
    x_max = layers.GlobalMaxPooling2D()(x)
    x_avg = layers.GlobalAveragePooling2D()(x)
    x = layers.Concatenate()([x_max, x_avg])
    x = layers.Dense(units=4, activation='softmax')(x)

    model = models.Model(inputs=resnet50.input, outputs=x, name='ResNet50_CBAM_Model')
    return model


# Model Intialization

In [ ]:
model = ResNet50_CBAM_Model()

# Compile models
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])

73506816/94765736 [======================>.......] - ETA: 1s

# Load original testing dataset (without augmentation)

In [8]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

# Load the testing dataset
test_dir = 'BrainTumor_MRI/Testing'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class names mapping (adjust if necessary)
class_names = test_dataset.classes

# To evaluate the trained model
# Save the confusion matrix + performance metrics

In [9]:
# Define class names
class_names = ["glioma", "meningioma", "notumor", "pituitary"]  # Modify according to your classes
MODEL_PATH = 'model_results_mu=0.0/'

# Function to evaluate the global model on the testing dataset
def evaluate_global_model(model, test_loader, output_file=f'{MODEL_PATH}Global_Model_Results_mu=0.0.txt'):
    all_preds = []
    all_labels = []
    total_loss = 0

    for features, labels in test_loader:
        # Convert to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        # Get predictions
        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)

        # Accumulate predictions and labels
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels_np)

        # Compute loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_tf, logits=predictions)
        total_loss += tf.reduce_mean(loss).numpy()

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Classification report
    classification_report_str = classification_report(all_labels, all_preds, target_names=class_names)

    # Append metrics to a file
    with open(output_file, "a") as file:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Get current timestamp
        file.write(f"Evaluation Timestamp: {timestamp}\n")
        file.write("Evaluation Metrics:\n")
        file.write(f"Accuracy: {accuracy * 100:.2f}%\n")
        file.write(f"Loss: {total_loss / len(test_loader):.4f}\n")
        file.write(f"Precision: {precision:.4f}\n")
        file.write(f"Recall: {recall:.4f}\n")
        file.write(f"F1 Score: {f1:.4f}\n\n")
        file.write("Classification Report:\n")
        file.write(classification_report_str)
        file.write("\nConfusion Matrix:\n")
        file.write("\n".join(["\t".join(map(str, row)) for row in conf_matrix]))
        file.write("\n" + "-" * 50 + "\n")

    # Print metrics
    print("\nClassification Report:\n", classification_report_str)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    current_loss = (total_loss / len(test_loader))
    
    # Plot and save heatmap
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    heatmap_path = f'model_results_mu=0.0/confusion_matrix_{timestamp.replace(" ", "_").replace(":", "-")}.png'
    plt.savefig(heatmap_path)  # Save as PNG with timestamp
    plt.close()  # Close the figure
    
    return current_loss


# FedProx hyper parameters

In [10]:
# Initial parameters
n_iter = 100
rounds_per_segment = 10
epochs = 1

mu & learning rate

In [11]:
lr = 0.0001
mu = 0.0

In [12]:
# Initialize lists to store cumulative training and testing metrics
all_train_loss = []
all_train_acc = []
all_test_loss = []
all_test_acc = []
# Load the initial model
# Trained_MODEL_PATH = ''
# model = tf.keras.models.load_model(Trained_MODEL_PATH)

# 100 rounds training

In [15]:
# Initialize early stopping parameters
patience = 20
best_metric = 0.9087  # Track the lowest validation loss
no_improve_rounds = 0  # Counter for rounds without improvement

In [16]:
model = tf.keras.models.load_model('model_results_mu=0.0/global_model_rounds_80_mu=0.0.h5')

In [17]:
# Function for FedProx training in segments
for i in range(80, n_iter, rounds_per_segment):
    print(f"Starting rounds {i+1} to {i+rounds_per_segment}")
    
    # Train for the specified number of rounds in this segment
    model, train_loss_f, train_acc_f, test_loss_f, test_acc_f = FedProx(
        model, Tumor_iid_train_dls, rounds_per_segment, Tumor_iid_test_dls, mu=mu, epochs=epochs, lr=lr
    )
    
    # Append the metrics to the cumulative lists
    all_train_loss.extend(train_loss_f)
    all_train_acc.extend(train_acc_f)
    all_test_loss.extend(test_loss_f)
    all_test_acc.extend(test_acc_f)
    
    print(f"Evaluating global model for rounds {i+1} to {i+rounds_per_segment}")
    current_metric = evaluate_global_model(model, test_loader, output_file=f'{MODEL_PATH}Global_Model_Results_mu={mu}.txt')
    
    # Check for early stopping based on validation loss
    # current_metric = test_loss_f[-1][-1]  # Use the most recent validation loss
    print(f"Current validation loss: {current_metric:.4f}")
    
    if best_metric is None or current_metric < best_metric:  # Improvement check
        best_metric = current_metric
        no_improve_rounds = 0  # Reset counter
        # Save the best model
        best_model_path = f'{MODEL_PATH}_{i+rounds_per_segment}th_round_best_model_mu={mu}.h5'
        model.save(best_model_path)
        print(f"Best model updated and saved at: {best_model_path}")
    else:
        no_improve_rounds += 10
        print(f"No improvement for {no_improve_rounds} rounds. Best validation loss: {best_metric:.4f}")

    # Save the model at the end of each segment
    segment_model_path = f'{MODEL_PATH}global_model_rounds_{i+rounds_per_segment}_mu={mu}.h5'
    model.save(segment_model_path)
    print(f"Model saved at: {segment_model_path}")
    
    # Test the saved global model on the testing dataset
    
    # Save the data to the file
    output_file = f'{MODEL_PATH}results-mu={mu}_{len(train_loss_f)}_rounds.txt'
    save_history_to_file(output_file, train_loss_f, train_acc_f, test_loss_f, test_acc_f)
    
    if no_improve_rounds >= patience:  # Early stopping condition
        print("Early stopping triggered. Training stopped.")
        break
    
    # Reload the model to ensure continuity for the next segment
    model = tf.keras.models.load_model(segment_model_path)


Starting rounds 81 to 90
Clients' weights: [0.22620689655172413, 0.2303448275862069, 0.2868965517241379, 0.25655172413793104]


2024-12-09 07:12:13.456942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2024-12-09 07:12:16.544847: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x1e30ac90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-09 07:12:16.544879: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-12-09 07:12:16.620297: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


====> i: 1 Server Test Accuracy: [80.47619047619048, 74.76190476190476, 81.66666666666667, 94.66666666666667]
====> i: 2 Server Test Accuracy: [78.33333333333333, 71.30952380952381, 78.92156862745098, 92.83333333333333]
====> i: 3 Server Test Accuracy: [77.97619047619048, 71.66666666666667, 78.33333333333333, 92.16666666666667]
====> i: 4 Server Test Accuracy: [75.35714285714286, 65.23809523809524, 69.70588235294117, 86.33333333333333]
====> i: 5 Server Test Accuracy: [78.69047619047619, 72.73809523809524, 80.0, 93.33333333333333]
====> i: 6 Server Test Accuracy: [82.02380952380952, 75.35714285714286, 83.82352941176471, 93.83333333333333]
====> i: 7 Server Test Accuracy: [80.0, 73.92857142857143, 81.76470588235294, 93.5]
====> i: 8 Server Test Accuracy: [78.21428571428571, 73.80952380952381, 80.19607843137256, 94.33333333333333]
====> i: 9 Server Test Accuracy: [80.0, 74.04761904761905, 81.96078431372548, 94.5]
====> i: 10 Server Test Accuracy: [80.47619047619048, 74.4047619047619, 82.

In [ ]:
# Save the cumulative history to a file after all rounds are complete
train_loss = all_train_loss[-100:]  # Adjusted to reflect 30 rounds for rounds 51 to 80
train_acc = all_train_acc[-100:]
test_loss = all_test_loss[-100:]
test_acc = all_test_acc[-100:]

# File to save the results
output_file = f'{MODEL_PATH}results-mu={mu}_{len(train_loss)}_rounds.txt'

# Save the data to the file
save_history_to_file(output_file, train_loss, train_acc, test_loss, test_acc)


# Load the trained model to test it

In [ ]:
mu=0.0
MODEL_PATH='model_results_mu=0.0/'
PATH = f'{MODEL_PATH}global_model_rounds_100_mu={mu}.h5'
model = tf.keras.models.load_model(PATH)

In [ ]:
evaluate_global_model(model, test_loader)

# to check how good the model to predict the class of a random image from testing data

In [ ]:
# Define class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Function to predict the class of a single image
def predict_image(image_path, model):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize to 224x224
    image_array = img_to_array(image)  # Convert to numpy array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = preprocess_input(image_array)  # Normalize for VGG19

    # Perform inference
    predictions = model(image_array, training=False)  # Inference mode
    probabilities = tf.nn.softmax(predictions[0]).numpy()  # Apply softmax
    predicted_class = np.argmax(probabilities)

    # Print the predicted class and probabilities
    print(f"Predicted Class: {class_names[predicted_class]}")
    print(f"Probabilities: {probabilities}")


In [ ]:
image_path = 'BrainTumor_MRI/Testing/meningioma/Te-me_0073.jpg'
predict_image(image_path, model)